# RL

## Pre-Knowledge

### Definition

- **Return**
  $$
  G_t=\Sigma_{t+1}^{\infty} \gamma R
  $$
  未来的折扣回报

- **State-Value**
  $$
  V_{\pi}(s)=\mathbb{E}[G_t|S_t=s]
  $$
  状态s下Return的期望

- **Action-Value** 
  $$
  Q_\pi(s, a)=\mathbb{E}[G_t|S_t=s,A_t=a]
  $$
  在状态 $s$ 下，选择动作a之后Return的期望

- **State-Value** 和 **Action-Value** 的关系
  $$
  \begin{aligned}
  V_\pi(s) &= \Sigma_{a \in \mathcal{A}} \mathbb{E}[G_t|S_t=s,A_t=a]\pi(a|s)\\
  &= \Sigma_{a \in \mathcal{A}}\pi(a|s)Q_\pi(s,a) \\
  &= \mathbb{E}_{A_t \in \pi(s)}[Q_\pi(s,A_t)]
  \end{aligned}
  $$
  即state-value为该状态下对应的动作值的期望

  由Bellman-Equation：
  $$
  \begin{aligned}
  V_\pi(s) &= 
  \underbrace{\Sigma_{a \in \mathcal{A}}\pi(a|s)\Sigma_{r \in \mathcal{R}}p(r|s,a)r}_{\text{即时奖励期望}}
  + \gamma\underbrace{\Sigma_{a \in \mathcal{A}}\pi(a|s)\Sigma_{s' \in \mathcal{S}}p(s'|s,a)V_\pi(s')}_{\text{未来奖励期望}}\\
  &=V_\pi(s)=\Sigma_{a \in \mathcal{A}}\pi(a|s)[\Sigma_{r \in \mathcal{R}}p(r|s,a)r+\gamma\Sigma_{s'\in \mathcal{S}}p(s'|s,a)V_\pi(s')] \\
  => Q_\pi(s,a)&=\Sigma_{r \in \mathcal{R}}p(r|s,a)r+\gamma\Sigma_{s'\in \mathcal{S}}p(s'|s,a)V_\pi(s')\\
  &=\mathbb{E}[R_{r+1}+\gamma V_\pi(S_{t+1})|S_t=s,A_t=a]
  \end{aligned}
  $$

- **On-Policy & Off-Policy**

  - On-Policy: 采样策略和目标策略一致
  - Off-Policy: 反之

### Stochastic approximation

- *RM*
  $$
  Object: g(w)=0
  $$
  对于 $g(w)$ 的观测值可能包含噪声
  $$
  \widetilde g(w,\eta)=g(w)+\eta
  $$
  求解 $w$:
  $$
  w_{k+1}=w_k-a_k\widetilde{g}(w_k,\eta_k)
  $$

### Temporal Difference

$$
\begin{aligned}
V_{t+1}(s_t)&=V_t(s_t)-\alpha_t(s_t)[V_t(s_t)-Target] \\
v_{t+1}(s_t)&=v_t(s_t), 当 s \neq s_t
\end{aligned}
$$



|  Algorithm  |                    Target                    |
| :---------: | :------------------------------------------: |
|    Sarsa    |    $r_{t+1}+\gamma q_t(s_{t+1},a_{t+1})$     |
| Q-learning  |     $r_{t+1}+\gamma max_aq_t(s_{t+1},a)$     |
| Monte Carlo | $r_{t+1}+\gamma r_{t+2}+\gamma^2r_{t+3}+...$ |





### Policy Gradient

定义最优策略

- 平均状态值
  $$
  \begin{aligned}
  \bar{v}_\pi&=\Sigma_{s \in \mathcal{S}}d(s)v_\pi(s) \\
  &=\mathbb{E}_{S\sim d}[v_\pi(S)] \\
  &=\Sigma_{s\in S}d(s)\mathbb{E}[\Sigma_{t=0}^{\infty}\gamma R_{t+1}|S_0=s]
  \end{aligned}
  $$

梯度
$$
\begin{aligned}
\nabla_{\theta}J(\theta)&=\Sigma_{s\in S}d(s)\Sigma_{a\in \mathcal{A}}\color{red}\nabla_{\theta}\pi_{\theta}(a|s)\color{black}q_\pi(s,a) \\
&=\Sigma_{s\in S}d(s)\Sigma_{a\in \mathcal{A}}\color{red}\nabla_{\theta}\log{\pi_{\theta}(a|s)}\pi_{\theta}(a|s)\color{black}q_\pi(s,a) \\
&=\mathbb{E}_{S\sim d,A\sim \pi_{\theta}(s)}[\nabla_{\theta}\log{\pi_\theta(A|S)q_\pi(S,A)}]
\end{aligned}
$$

- REINFORCE(使用蒙特卡洛方法估计$q_t$)

  Stochastic Gradient
  $$
  \theta_{t+1}=\theta_t+\alpha_{\theta}\nabla_{\theta}\log{\pi_\theta(A|S)q_t(S,A)} \\
  $$

- 优势函数
  $$
  \mathbb{E}_{S\sim d,A\sim \pi_{\theta}(s)}[\nabla_{\theta}\log{\pi_\theta(A|S)(\color{red}q_\pi(S,A)-v_\pi(S)\color{black})}]
  $$
  我们不关心在$s_t$下选择$a_t$带来的绝对回报，而是计算其相对于其他动作的**优势**。

  动作的好坏是相对的。

### Importance Sampline

$$
\mathbb{E}_{X \sim p_0}[X]=\int p_0(x)xdx=\int p_1(x)\frac{p_0(x)}{p_1(x)}xdx=\int p_1(x)f(x)dx=\mathbb{E}_{X\sim p1}[f(X)]
$$

使用$p_1$分布近似$p_0$, 这样，可以将`Actor-Critic`转为Off-Policy的版本

使用重要性采样后，梯度为：
$$
\nabla J(\theta)=\mathbb{E}[\frac{\pi_\theta (a_t|s_t)}{\pi_{\theta _{odd}}(a_t|s_t)}A_{\phi}(s_t,a_t)\nabla \log{\pi_\theta(a_t|s_t)}]
$$

### GAE：平衡方差和偏差

$$
A_\pi(s_t, a_t)=\mathbb{E}_{s_t+1\sim P(·|s_t,a_t)}[r_t+\gamma V_\pi(s_t+1)-V_\pi(s_t)]
$$

当 $V_\pi$ 估计不准时，$r_t+\gamma V_\pi(s_t+1)-V_\pi(s_t)$ 偏差较大，将其展开
$$
r_t+\gamma V_\pi(s_t+1)-V_\pi(s_t)=\Sigma_{l=0}^{\infin}\gamma ^lr_{t+l}-V_\pi(s_t)
$$
偏差虽然降低了，但是方差很大 ($Var(r_t)=>Var(r_t+...)$)

需要采样足够多的数据才能估计出优势函数
$$
\begin{aligned}
A_t^{(1)} &= \delta_t &= -V(s_t) + r_t + \gamma V(s_{t+1}) \\
A_t^{(2)} &= \delta_t + \gamma \delta_{t+1} &= -V(s_t) + r_t + \gamma r_{t+1} + \gamma^2 V(s_{t+2}) \\
A_t^{(k)}&=\Sigma_{l=0}^{k-1}\gamma^l\delta_{t+1}
\end{aligned}
$$

$$
\begin{aligned}
A_t^{GAE(\gamma,\lambda)}&=(1-\lambda)(A_t^{(1)}+\lambda A_t^{(2)}+\lambda^2 A_t^{(3)}+...)\\
&=\Sigma_{l=0}^\infin(\gamma\lambda)^l\delta_{t+l}
\end{aligned}
$$

- $\lambda=1$时，退化为MC（**高方差，低偏差**）
- $\lambda=0$时，退化为1步TD（**低方差，高偏差**）

## TRPO

引入 `GAE` 解决单步优势的方差-偏差平衡后，优化目标变为：
$$
argmax_{\pi_\theta}J(\pi_\theta)=\mathbb{E_{\tau\sim\pi_{old}}}[\frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}A_\phi^{GAE}(s_t,a_t)]
$$
当使用重要性采样估计 $J(\pi_\theta)$, 如果两个分布之间的差异过大，估计是不准确的。`TRPO` 将两个分布的 `KL` 散度作为约束条件，
$$
subject to \mathbb{E}[KL(\pi_{\theta_{old}},\pi_{\theta})]<\delta
$$


但是不将限制直接加在损失函数中，优化过程较为复杂

## PPO

1. PPO将`KL`散度作为一个损失项：

$$
argmax_{\pi_\theta}J(\pi_\theta)=\mathbb{E_{\tau\sim\pi_{old}}}[\frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}A_\phi^{GAE}(s_t,a_t)-\beta KL(\pi_{\theta_{old}},\pi_{\theta})]
$$

2. CLIP

   将$r_t(\theta)=\frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}$限制在$[1-\epsilon,1+\epsilon]$之间
   $$
   argmax_{\pi_\theta}J(\pi_\theta)=\mathbb{E_{\tau\sim\pi_{old}}}[min(r_t(\theta)A_\phi^{GAE}(s_t,a_t),clip(r_t(\theta), 1-\epsilon,1+\epsilon)A_\phi^{GAE}(s_t,a_t))]
   $$

$\Theta$